## How to create a classifier model form the nuclei features data

This tutorial will create a *Random forest* algorithm based nuclei classifier using features extracted from an image data.

*This tutorial is created by Subin Erattakulangara (Kitware)*

### Step 1

Download a sample image file from : <br>
Open the Nuclei Feature extraction panel in DSA. and upload the image data into area shown below. provide the location for both feature file and annotation files to be saved. then press submit to start the process.The cli will generate the feature file requried for the classifier. Annotation file is not required for creating the classfier.


### Step

Download the feature files once the cli generates it and store it in a different folder. Here is the output feature file generated for the image given above. Run the following python code inside the folder to create a feature file with classes.